## Deliverable 3. Create a Travel Itinerary Map.

In [23]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import csv

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [24]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.

vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv")
vacation_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,0,Saint-Leu,RE,70.63,scattered clouds,-21.1500,55.2833,La Caz Oceane
1,2,Sao Joao Da Barra,BR,73.09,clear sky,-21.6403,-41.0511,Pousada Porto De Canoas
2,6,Vanimo,PG,81.61,overcast clouds,-2.6741,141.3028,Vahmoneh Lodge Vanimo
3,10,Lompoc,US,70.57,broken clouds,34.6391,-120.4579,"Holiday Inn Express Lompoc, an IHG Hotel"
4,13,Rikitea,PF,76.26,scattered clouds,-23.1203,-134.9692,Pension Maro'i


In [88]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]
max_temp = vacation_df["Max Temp"]

In [89]:
# 4a. Add a marker layer for each city to the map.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating = False,
                                max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 4b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))

In [90]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# I was going to visit Mexico for the Day of the Dead, because of covid, everything seized.
# so I choose Topolobampo, Las Choapas, Ixtapa, La Cruz, Topolobampo for a roundtrip.
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

vacation_start = vacation_df.loc[vacation_df['City']=='Topolobampo']
vacation_end = vacation_df.loc[vacation_df["City"]=='Topolobampo']
vacation_stop1 = vacation_df.loc[vacation_df['City']=='Las Choapas']
vacation_stop2 = vacation_df.loc[vacation_df['City']=='Ixtapa'] 
vacation_stop3 = vacation_df.loc[vacation_df['City']=='La Cruz'] 
#vacation_start


In [91]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = vacation_start['Lat'].to_numpy()[0], vacation_start['Lng'].to_numpy()[0]
end = vacation_end['Lat'].to_numpy()[0], vacation_end['Lng'].to_numpy()[0]
stop1 = vacation_stop1['Lat'].to_numpy()[0], vacation_stop1['Lng'].to_numpy()[0]
stop2 = vacation_stop2['Lat'].to_numpy()[0], vacation_stop2['Lng'].to_numpy()[0]
stop3 = vacation_stop3['Lat'].to_numpy()[0], vacation_stop3['Lng'].to_numpy()[0]



In [92]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
fig=gmaps.figure(map_type= 'HYBRID')
vacation_itineary = gmaps.directions_layer(start, end, waypoints=[stop1, stop2, stop3], 
                                          travel_mode = 'BICYCLING', show_markers=True,
                                          show_route=True)
fig.add_layer(vacation_itineary)
fig

Figure(layout=FigureLayout(height='420px'))

In [93]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
vacation_itinerary = pd.concat([vacation_start, vacation_stop1,vacation_stop2,vacation_stop3],
                               ignore_index=True, sort=False)
vacation_itinerary

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,593,Topolobampo,MX,89.06,scattered clouds,25.6000,-109.0500,Hotel Marina Topolobampo
1,603,Las Choapas,MX,84.25,scattered clouds,17.9333,-94.0833,Hotel Jysmak
2,381,Ixtapa,MX,81.23,broken clouds,20.7000,-105.2000,Hotel Plaza Providencia
3,325,La Cruz,MX,84.47,overcast clouds,23.9167,-106.9000,LIBERADOR DE CELULAS MADRE ADULTAS


In [94]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
itinerary_hotel_info = [info_box_template.format(**row) for index, row in vacation_itinerary.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_itinerary[["Lat", "Lng"]]

In [95]:
print(locations)

       Lat       Lng
0  25.6000 -109.0500
1  17.9333  -94.0833
2  20.7000 -105.2000
3  23.9167 -106.9000


In [97]:
# 11a. Add a marker layer for each city to the map.
fig=gmaps.figure(map_type= 'HYBRID')
marker_layer=gmaps.marker_layer(locations, info_box_content=itinerary_hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig 

Figure(layout=FigureLayout(height='420px'))